In [1]:
# %pip install jupysql trino pandas pyarrow --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install sqlalchemy-trino --quiet


[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

df = pd.read_parquet(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet"
)
df.shape

(1369769, 19)

In [19]:
df.head(1000)

VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count   
0           1  2021-01-01 00:30:10   2021-01-01 00:36:12              1.0  \
1           1  2021-01-01 00:51:20   2021-01-01 00:52:19              1.0   
2           1  2021-01-01 00:43:30   2021-01-01 01:11:06              1.0   
3           1  2021-01-01 00:15:48   2021-01-01 00:31:01              0.0   
4           2  2021-01-01 00:31:49   2021-01-01 00:48:21              1.0   
..        ...                  ...                   ...              ...   
995         1  2021-01-01 00:57:42   2021-01-01 01:08:18              1.0   
996         2  2021-01-01 00:19:34   2021-01-01 00:31:34              1.0   
997         2  2021-01-01 00:16:38   2021-01-01 00:24:23              3.0   
998         2  2021-01-01 00:28:09   2021-01-01 00:40:08              1.0   
999         2  2021-01-01 00:28:09   2021-01-01 00:40:08              1.0   

     trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID   
0             2.10         1.0                  N           142            43  \
1             0.20         1.0                  N           238           151   
2            14.70         1.0                  N           132           165   
3            10.60         1.0                  N           138           132   
4             4.94         1.0                  N            68            33   
..             ...         ...                ...           ...           ...   
995           3.10         1.0                  N           264           264   
996           2.77         1.0                  N           170           239   
997           2.01         1.0                  N            41           263   
998           3.35         1.0                  N           263           186   
999           3.35         1.0                  N           263           186   

     payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount   
0               2          8.0    3.0      0.5        0.00           0.0  \
1               2          3.0    0.5      0.5        0.00           0.0   
2               1         42.0    0.5      0.5        8.65           0.0   
3               1         29.0    0.5      0.5        6.05           0.0   
4               1         16.5    0.5      0.5        4.06           0.0   
..            ...          ...    ...      ...         ...           ...   
995             1         11.5    0.5      0.5        1.00           0.0   
996             1         11.0    0.5      0.5        2.96           0.0   
997             1          8.5    0.5      0.5        2.46           0.0   
998             4        -12.0   -0.5     -0.5        0.00           0.0   
999             2         12.0    0.5      0.5        0.00           0.0   

     improvement_surcharge  total_amount  congestion_surcharge  airport_fee  
0                      0.3         11.80                   2.5          NaN  
1                      0.3          4.30                   0.0          NaN  
2                      0.3         51.95                   0.0          NaN  
3                      0.3         36.35                   0.0          NaN  
4                      0.3         24.36                   2.5          NaN  
..                     ...           ...                   ...          ...  
995                    0.3         13.80                   0.0          NaN  
996                    0.3         17.76                   2.5          NaN  
997                    0.3         14.76                   2.5          NaN  
998                   -0.3        -15.80                  -2.5          NaN  
999                    0.3         15.80                   2.5          NaN  

[1000 rows x 19 columns]

In [1]:
from sqlalchemy import create_engine

engine = create_engine(
    "trino://user@trino:8080/example", connect_args={"user": "user"}
)

# df.to_sql(
#     con=engine,
#     name="taxi_2",
#     schema="default",
#     method="multi",
#     index=False,
# )

engine.dispose()

In [11]:
%load_ext sql

In [13]:
%sql trino://user@trino:8080/demo


Connecting to 'trino://user@trino:8080/demo'

In [14]:
%%sql
SHOW SCHEMAS

Running query in 'trino://user@trino:8080/demo'

+--------------------+
|       Schema       |
+--------------------+
| information_schema |
|        nyc         |
+--------------------+

In [15]:
%%sql 
USE demo.nyc

Running query in 'trino://user@trino:8080/demo'

++
||
++
++

In [16]:
%%sql
SHOW TABLES

Running query in 'trino://user@trino:8080/demo'

+-------+
| Table |
+-------+
| taxis |
+-------+

In [17]:
%%sql
SELECT * FROM taxis LIMIT 10

Running query in 'trino://user@trino:8080/demo'

+----------+---------------------------+---------------------------+-----------------+---------------+------------+--------------------+--------------+--------------+--------------+-------------+-------+---------+------------+--------------+-----------------------+--------------+----------------------+-------------+
| vendorid |    tpep_pickup_datetime   |   tpep_dropoff_datetime   | passenger_count | trip_distance | ratecodeid | store_and_fwd_flag | pulocationid | dolocationid | payment_type | fare_amount | extra | mta_tax | tip_amount | tolls_amount | improvement_surcharge | total_amount | congestion_surcharge | airport_fee |
+----------+---------------------------+---------------------------+-----------------+---------------+------------+--------------------+--------------+--------------+--------------+-------------+-------+---------+------------+--------------+-----------------------+--------------+----------------------+-------------+
|    2     | 2022-01-09 09:24:06+00:00 | 2022-01-09 09:45:35+00:00 |       1.0       |      9.81     |    1.0     |         N          |     138      |     244      |      2       |     29.0    |  0.0  |   0.5   |    0.0     |     6.55     |          0.3          |     37.6     |         0.0          |     1.25    |
|    2     | 2022-01-09 11:27:14+00:00 | 2022-01-09 11:47:43+00:00 |       1.0       |      9.01     |    1.0     |         N          |     138      |     230      |      2       |     27.5    |  0.0  |   0.5   |    0.0     |     6.55     |          0.3          |     38.6     |         2.5          |     1.25    |
|    2     | 2022-01-09 13:14:53+00:00 | 2022-01-09 13:23:52+00:00 |       1.0       |      3.52     |    1.0     |         N          |     138      |      7       |      1       |     12.0    |  0.0  |   0.5   |    2.81    |     0.0      |          0.3          |    16.86     |         0.0          |     1.25    |
|    2     | 2022-01-09 00:00:18+00:00 | 2022-01-09 00:12:53+00:00 |       1.0       |      4.92     |    1.0     |         N          |     114      |      75      |      1       |     15.0    |  0.5  |   0.5   |    3.76    |     0.0      |          0.3          |    22.56     |         2.5          |     0.0     |
|    2     | 2022-01-09 00:00:46+00:00 | 2022-01-09 00:13:20+00:00 |       1.0       |      3.04     |    1.0     |         N          |     140      |     246      |      1       |     11.5    |  0.5  |   0.5   |    2.5     |     0.0      |          0.3          |     17.8     |         2.5          |     0.0     |
|    2     | 2022-01-09 00:00:44+00:00 | 2022-01-09 00:05:42+00:00 |       1.0       |      1.37     |    1.0     |         N          |     141      |     162      |      1       |     6.0     |  0.5  |   0.5   |    1.2     |     0.0      |          0.3          |     11.0     |         2.5          |     0.0     |
|    2     | 2022-01-09 13:51:42+00:00 | 2022-01-09 14:16:59+00:00 |       1.0       |      10.1     |    1.0     |         N          |     138      |     162      |      2       |     29.0    |  0.5  |   0.5   |    0.0     |     6.55     |          0.3          |     40.6     |         2.5          |     1.25    |
|    2     | 2022-01-09 15:09:41+00:00 | 2022-01-09 15:38:33+00:00 |       1.0       |     11.76     |    1.0     |         N          |     138      |     181      |      2       |     35.5    |  0.5  |   0.5   |    0.0     |     0.0      |          0.3          |    38.05     |         0.0          |     1.25    |
|    2     | 2022-01-09 17:05:34+00:00 | 2022-01-09 17:36:28+00:00 |       1.0       |     12.86     |    1.0     |         N          |     138      |      37      |      2       |     36.0    |  0.5  |   0.5   |    0.0     |     0.0      |          0.3          |    38.55     |         0.0          |     1.25    |
|    2     | 2022-01-09 00:02:12+00:00 | 2022-01-09 00:11:44+00:00 |       1.0       |      2.43     |    1.0     |         N          |      48      |      90      |      2       |   

In [19]:
%%sql
SELECT * FROM "taxis$snapshots" ORDER BY committed_at desc

Running query in 'trino://user@trino:8080/demo'

+----------------------------------+---------------------+-----------+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|           committed_at           |     snapshot_id     | parent_id | operation |                                                                  manifest_list                                                                  |                                                                                                                                                                    summary                                                                                                                                                                    |
+----------------------------------+---------------------+-----------+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| 2024-07-21 13:43:51.838000+00:00 | 5318660823333782471 |    None   |   append  | s3a://warehouse/wh/nyc/taxis_c024c2a8-f1a8-46ea-8df9-d98ee2749b2f/metadata/snap-5318660823333782471-1-0f337b83-d47b-4216-83b3-f71dc65df3f1.avro | {'spark.app.id': 'local-1721569408198', 'changed-partition-count': '40', 'added-data-files': '40', 'total-equality-deletes': '0', 'added-records': '2463931', 'total-position-deletes': '0', 'added-files-size': '40057638', 'total-delete-files': '0', 'total-files-size': '40057638', 'total-records': '2463931', 'total-data-files': '40'} |
+----------------------------------+---------------------+-----------+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+